In [ ]:
!rm -R AML2021
!git clone https://github.com/sleuoth-hof/AML2021.git

In [ ]:
_file_dir = './AML2021/emotion-recognition/'

In [ ]:
_kafka_sever = ''

In [ ]:
from tensorflow.keras.models import model_from_json
import numpy as np
import tensorflow as tf

class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy", "Neutral","Sad","Surprised"]
    
    def __init__(self, model_json_file, model_weights_file):
        with open(model_json_file,"r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)

        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model.make_predict_function()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]

In [ ]:
import cv2
import numpy as np

facec = cv2.CascadeClassifier(_file_dir + 'haarcascade_frontalface_default.xml')
model = FacialExpressionModel(_file_dir + "model.json", _file_dir + "model_weights.h5")


In [ ]:
!pip install kafka-python

In [ ]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
import base64
from google.colab.patches import cv2_imshow


producer = KafkaProducer(bootstrap_servers=_kafka_sever)

consumer = KafkaConsumer('faces_found_detail', bootstrap_servers=[_kafka_sever], auto_offset_reset='latest')


for message in consumer:

 print(message.value)

 data = json.loads(message.value)

 img = base64.b64decode(data["image_cut_out"]); 

 
 npimg = np.fromstring(img, dtype=np.uint8); 
 fr = cv2.imdecode(npimg, 1)

 cv2_imshow(fr)

 gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)

 (x,y,w,h) = data["object_box"]

 #fc = gray_fr[y:y+h, x:x+w]

 cv2_imshow(gray_fr)

 roi = cv2.resize(gray_fr, (48, 48))
 pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

 print(pred)

 j = {
     "id": data["id"],
     "face_nr": data["face_nr"],
     "box": [x, y, w, h],
     "emotion": pred,
     "image_cut_out": data["image_cut_out"] 
 }

 print(str.encode(json.dumps(j)))

 producer.send('gui-response_emotion', str.encode(json.dumps(j)))